#Котелевский БПМ-19-2

In [ ]:
from typing import List
from math import log2, ceil
from random import randrange
import binascii

In [ ]:
def __hamming_common(src: List[List[int]], s_num: int, encode=True) -> int:
    s_range = range(s_num)
    errors = 0
    for i in src:
        sindrome = 0
        for s in s_range:
            sind = 0
            for p in range(2 ** s, len(i) + 1, 2 ** (s + 1)):
                for j in range(2 ** s):
                    if (p + j) > len(i):
                        break
                    sind ^= i[p + j - 1]
            if encode:
                i[2 ** s - 1] = sind
            else:
                sindrome += (2 ** s * sind)
        if (not encode) and sindrome:
            try:
                i[sindrome - 1] = int(not i[sindrome - 1])
            except IndexError:
                errors += 1
    return errors

In [ ]:
def crc32(data):
    data = data.encode('utf-8')
    crc = 0xFFFFFFFF
    poly = 0x04C11DB7

    for byte in data:
        crc ^= byte
        for _ in range(8):
            if crc & 1:
                crc = (crc >> 1) ^ poly
            else:
                crc >>= 1

    return crc ^ 0xFFFFFFFF

In [ ]:
def hamming_encode(msg: str, mode: int = 8) -> str:
    """
    Encoding the message with Hamming code.
    :param msg: Message string to encode
    :param mode: number of significant bits
    :return: 
    """

    result = ""

    msg_b = msg.encode("utf8")
    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    bit_seq = []
    for byte in msg_b:  # get bytes to binary values; every bits store to sublist
        bit_seq += list(map(int, f"{byte:08b}"))

    res_len = ceil((len(msg_b) * 8) / mode)     # length of result (bytes)
    bit_seq += [0] * (res_len * mode - len(bit_seq))    # filling zeros

    to_hamming = []

    for i in range(res_len):    # insert control bits into specified positions
        code = bit_seq[i * mode:i * mode + mode]
        for j in range(s_num):
            code.insert(2 ** j - 1, 0)
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, True)   # process

    for i in to_hamming:
        result += "".join(map(str, i))

    return result

In [ ]:
def hamming_decode(msg: str, mode: int = 8):
    """
    Decoding the message with Hamming code.
    :param msg: Message string to decode
    :param mode: number of significant bits
    :return: 
    """

    result = ""

    s_num = ceil(log2(log2(mode + 1) + mode + 1))   # number of control bits
    res_len = len(msg) // (mode + s_num)    # length of result (bytes)
    code_len = mode + s_num     # length of one code sequence

    to_hamming = []

    for i in range(res_len):    # convert binary-like string to int-list
        code = list(map(int, msg[i * code_len:i * code_len + code_len]))
        to_hamming.append(code)

    errors = __hamming_common(to_hamming, s_num, False)  # process

    for i in to_hamming:    # delete control bits
        for j in range(s_num):
            i.pop(2 ** j - 1 - j)
        result += "".join(map(str, i))

    msg_l = []

    for i in range(len(result) // 8):   # convert from binary-sring value to integer
        val = "".join(result[i * 8:i * 8 + 8])
        msg_l.append(int(val, 2))

    # finally decode to a regular string
    try:
        result = bytes(msg_l).decode("utf-8")
    except UnicodeDecodeError:
        pass

    return result, errors

In [ ]:
def noizer(msg: str, mode: int) -> str:
    """
    Generates an error in each element of a Hamming encoded message
    """
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize = randrange(code_len)
        to_noize[noize] = int(not to_noize[noize])
        result += "".join(map(str, to_noize))

    return result

In [ ]:
def noizer4(msg: str, mode: int) -> str:
    """
    Generates up to 4 errors in each element of a Hamming encoded message
    """
    seq = list(map(int, msg))
    s_num = ceil(log2(log2(mode + 1) + mode + 1))  # количество служебных битов
    code_len = mode + s_num  # длина кодового слова
    cnt = len(msg) // code_len
    result = ""

    for i in range(cnt):
        to_noize = seq[i * code_len:i * code_len + code_len]
        noize1 = randrange(code_len)
        noize2 = randrange(code_len)
        noize3 = randrange(code_len)
        noize4 = randrange(code_len)
        to_noize[noize1] = int(not to_noize[noize1])
        to_noize[noize2] = int(not to_noize[noize2])
        to_noize[noize3] = int(not to_noize[noize3])
        to_noize[noize4] = int(not to_noize[noize4])
        result += "".join(map(str, to_noize))

    return result

In [ ]:
MODE = 63  # длина слова с контрольными битами составляет 73 => значащих битов в слове 66
msg = 'Формула-1 (англ. Formula One, Formula 1) — чемпионат мира по кольцевым гонкам, который проводится ежегодно и состоит из этапов (называемых Гран-при), в соответствии с техническими нормами, требованиями и правилами, установленными Международной автомобильной федерацией (FIA). Название происходит от набора технических правил (т. н. «формулы»), которому должны соответствовать автомобили всех участников[1]. С 1950 по 1980 год проводился чемпионат мира для гонщиков (англ. World Championship for Drivers), на смену которому в 1981 году пришёл чемпионат мира Формулы-1 (англ. Formula 1 World Championship), позже получивший современное название: FIA Formula One World Championship[2][3]. Турнир находится под управлением Международной автомобильной федерации, ответственной за продвижение на данный момент является группа компаний Formula One Group. Чемпионат мира проводится каждый год и состоит из отдельных этапов, имеющих статус Гран-при, в конце года подводятся итоги турнира, с учётом суммы набранных за сезон очков. В Формуле-1 соревнуются как отдельные гонщики (за титул чемпиона мира среди гонщиков, также называемый в русском языке личным зачётом (чемпионата мира)), так и отдельные команды (за титул чемпиона мира среди конструкторов, для которого в русском языке часто используется его историческое название «кубок конструкторов»). Техника участников должна соответствовать техническому регламенту Формулы-1 и пройти тест на ударопрочность. Команды, участвующие в гонках Формулы-1, используют на Гран-при гоночные автомобили собственного производства. Таким образом, задачей команды является не только нанять быстрого и опытного гонщика и обеспечить грамотную настройку и обслуживание машины, но и вообще самостоятельно спроектировать и сконструировать машину. Но бывают и исключения. Например, шасси команд Red Bull Racing и Scuderia Toro Rosso (с 2020 Scuderia Alpha Tauri) были очень похожи, почти идентичны, вплоть до 2009 года. Они были спроектированы и изготовлены компанией Red Bull Technology, так как обе команды и компания-изготовитель принадлежат концерну Red Bull GmbH[4]. Поскольку команды строят машины по собственным технологиям и ввиду высокой конкуренции команд, в Формуле-1 постоянно появляются оригинальные технические решения, что ведёт к прогрессу как гоночных машин, так и дорожных автомобилей[5].'
print(f'Сообщение:\n{msg}')
checksum = crc32(msg)
print(f'Контрольная сумма: {checksum}\n')

print('Первая отправка (без ошибок)\n')
enc_msg = hamming_encode(msg, MODE)
print(f'Кодированное сообщение:\n{enc_msg}')
dec_msg, err = hamming_decode(enc_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc32(dec_msg)}, корректность: {crc32(dec_msg) == checksum}\n')
print(f'MSG: {msg == dec_msg}')

print('Вторая отправка (не более 1 ошибки на слово)\n')
noize_msg = noizer(enc_msg, MODE)
print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
dec_msg, err = hamming_decode(noize_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc32(dec_msg)}, корректность: {crc32(dec_msg) == checksum}\n')
print(f'MSG: {msg == dec_msg}')

print('Третья отправка (4 ошибки на слово)\n')
noize_msg = noizer4(enc_msg, MODE)
print(f'Кодированное сообщение с ошибками:\n{noize_msg}')
dec_msg, err = hamming_decode(noize_msg, MODE)
dec_msg = dec_msg[:-2:]
print(f'Раскодированное сообщение:\n{dec_msg}')
print(
    f'Контрольная сумма: {crc32(dec_msg)}, корректность: {crc32(dec_msg) == checksum}, количество обнаруженных ошибок: {err}')

Сообщение:
Формула-1 (англ. Formula One, Formula 1) — чемпионат мира по кольцевым гонкам, который проводится ежегодно и состоит из этапов (называемых Гран-при), в соответствии с техническими нормами, требованиями и правилами, установленными Международной автомобильной федерацией (FIA). Название происходит от набора технических правил (т. н. «формулы»), которому должны соответствовать автомобили всех участников[1]. С 1950 по 1980 год проводился чемпионат мира для гонщиков (англ. World Championship for Drivers), на смену которому в 1981 году пришёл чемпионат мира Формулы-1 (англ. Formula 1 World Championship), позже получивший современное название: FIA Formula One World Championship[2][3]. Турнир находится под управлением Международной автомобильной федерации, ответственной за продвижение на данный момент является группа компаний Formula One Group. Чемпионат мира проводится каждый год и состоит из отдельных этапов, имеющих статус Гран-при, в конце года подводятся итоги турнира, с учётом 